# Lab 02 - Introduction to NLP

## The dataset

### 1. How many splits does the dataset has ?

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names("imdb")

There is three splits: train, test and unsupervised.

### 2. How big are these splits ?

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset

In [ ]:
dataset.num_rows

The train set has 25 000 rows, as the test set and the unsupervised has 50 000 rows.

### 3. What is the proportion of each class on the supervised splits ?

In [ ]:
dataset.num_columns

In [ ]:
dataset.column_names

In [ ]:
dataset.data['train']['label'].value_counts()

In [ ]:
dataset.data['test']['label'].value_counts()

The test and train sets are composed of half label 0 and half label 1.

## Naive Bayes classifier

#### Question 1

In [ ]:
import string

In [ ]:
def lower_and_remove_punctuation(dict_elt):
    '''
    dict_elt : Lower and remove the punctuation in an element of a dictionary of the dataset, it requires ['text']
    '''
    dict_elt['text'] = dict_elt['text'].lower().translate(str.maketrans('', '', string.punctuation))
    return dict_elt

In [ ]:
dataset_transform = dataset.map(lower_and_remove_punctuation)
dataset_transform.data['train']['text'][0]

#### Question 2

We have decided to use scikit-learn Pipeline with a CountVectorizer and MunltinomialNB classifier.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

In [24]:
pipe.fit(dataset_transform.data['train']['text'].to_numpy(), dataset_transform.data['train']['label'].to_numpy())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [25]:
prediction_train = pipe.predict(dataset_transform.data['train']['text'].to_numpy()) == dataset_transform.data['train']['label'].to_numpy()
prediction_test = pipe.predict(dataset_transform.data['test']['text'].to_numpy()) == dataset_transform.data['test']['label'].to_numpy()

#### Question 3

In [26]:
prediction_train.sum()/len(prediction_train)

0.91284

In [27]:

prediction_test.sum()/len(prediction_test)

0.8172

#### TODO Question 4 : Why is accuracy a sufficient measure of evaluation here ?  
    Because the given labels only has two options 0 for negative and 1 for positive ! well if you are that smart... just answer the question then XP

#### TODO BONUS

#### Question 6

In [98]:
list_index = [index for index,value in enumerate(prediction_test) if not value] # list of false element by prediction
# len(list_index) / len(prediction_test) check if we have the good amount of false answers

In [101]:
list_index_0 = [index for index,value in enumerate(dataset_transform.data['test']['label'].to_numpy()) if value == 0 and index in list_index]
list_index_1 = [index for index,value in enumerate(dataset_transform.data['test']['label'].to_numpy()) if value == 1 and index in list_index]

In [102]:
print("text: ", dataset_transform.data['test']['text'][list_index_0[0]])
print("label: ", dataset_transform.data['test']['label'][list_index_0[0]])

text:  blind date columbia pictures 1934 was a decent film but i have a few issues with this film first of all i dont fault the actors in this film at all but more or less i have a problem with the script also i understand that this film was made in the 1930s and people were looking to escape reality but the script made ann sotherns character look weak she kept going back and forth between suitors and i felt as though she should have stayed with paul kellys character in the end he truly did care about her and her family and would have done anything for her and he did by giving her up in the end to fickle neil hamilton who in my opinion was only out for a good time paul kellys character although a workaholic was a man of integrity and truly loved kitty ann sothern as opposed to neil hamilton while he did like her a lot i didnt see the depth of love that he had for her character the production values were great but the script could have used a little work
label:  0


In this example the model has probably failed because the comment is not fully negative, it said that it's 'decent', have some 'problem'...

In [103]:
print("text: ", dataset_transform.data['test']['text'][list_index_1[0]])
print("label: ", dataset_transform.data['test']['label'][list_index_1[0]])

text:  i admire deepa mehta and this movie is a masterpiece id recommend to buy this movie on dvd because its a movie you might want to watch more often than just once and trust me youd still find little meaningful details after watching it several timesbr br the characters  except for the grandmother perhaps  are all very balanced no black and white even though you follow the story from the perspective of the two protagonists there is also empathy for the other charactersbr br i think the imdb rating for the movie is far too low  probably due to its politically controversial content
label:  1


In this case the model classify wrong a text that was positive. 

## **Stemming** ~~and Lemmatization~~

#### Question 1

We choose stemming pretreatment

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
# We need to download a package for word tokenization
nltk.download('punkt')

In [ ]:
import re

def stemming_builder():
    re_word = re.compile(r"^\w+$")
    stemmer = SnowballStemmer("english")
    def wesh(text):
        stemmed = [stemmer.stem(word) for word in word_tokenize(text['text'].lower()) if re_word.match(word)]
        text['text'] = " ".join(stemmed)
        return text
    return wesh

In [ ]:
dataset_stemming = dataset.map(stemming_builder())
dataset_stemming.data['train']['text'][0]

#### Question 2

In [ ]:
pipe.fit(dataset_stemming.data['train']['text'].to_numpy(), dataset_stemming.data['train']['label'].to_numpy())

In [ ]:
prediction_train = pipe.predict(dataset_stemming.data['train']['text'].to_numpy()) == dataset_stemming.data['train']['label'].to_numpy()
prediction_test = pipe.predict(dataset_stemming.data['test']['text'].to_numpy()) == dataset_stemming.data['test']['label'].to_numpy()

In [ ]:
prediction_train.sum()/len(prediction_train)

In [ ]:
prediction_test.sum()/len(prediction_test)

#### Question 3

Results be like :  
![homer gif](homer.gif)